# RATING Project

Comic Mischief Detection Task

Files:

1. "train.csv" : 
-- Contains multiclass classification content annotations for each video scene used in the training set.
-- Annotations are on a scene level and do not correspond to a specific modality
-- a ".csv" file containing video URLs as well as the IDs of the scenes used in the training set.
-- Videos are available in the form of URLs, collected from the Youtube and the IMDB websites.
-- Contains metadata about the videos.
-- Four content categories related to comic mischief are used (Sarcasm, Slapstick Humor, Gory Humor, Mature Humor).

2. "val.csv" : 
-- Contains multiclass classification content annotations for each video scene used in the validation set.
-- You can use this set for performing model hyperparameter tuning before using the test set


3. "test.csv" : 
-- Contains multiclass classification content annotations for each video scene used in the test set.
-- You can use this set for evaluating your method

In [1]:
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow import keras
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

device_name = tf.test.gpu_device_name()
print("device name:", device_name)

tf.test.is_gpu_available()


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12723479156221979743
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6302793728
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6607330029377686525
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060 SUPER, pci bus id: 0000:02:00.0, compute capability: 7.5"
]
2.6.0
Num GPUs Available:  1
device name: /device:GPU:0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
# global variables

train_dir = os.getcwd() + "/train_data/"
val_dir = os.getcwd() + "/val_data/"
test_dir = os.getcwd() + "/test_data/"

# Hyperparameters
MAX_SEQ_LENGTH = 128
NUM_FEATURES = 1024
IMG_SIZE = 224

EPOCHS = 10
BATCH_SIZE = 64

### References
1. https://keras.io/examples/vision/video_transformers/)
2. https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub)
3. https://colab.research.google.com/github/sayakpaul/Action-Recognition-in-TensorFlow/blob/main/Data_Preparation_UCF101.ipynb

## Data Preparation (not complete)

### METADATA loading

In [4]:
# Create a dataframe which contains multiclass classification content annotations for each video scene used in the training set.
train_df = pd.read_csv('train-updated.csv', dtype={'combination': object}).iloc[:,1:]
train_df["path"] = train_dir + train_df["Video ID"]+ ".0" + train_df["Scene_ID"].astype(str) + ".mp4"
train_df.head()

,Video ID,Scene_ID,Video URL,Codec,Resolution,Avg Frame rate,Mature Humor - Scene Annotation,Slapstick Humor - Scene Annotation,Gory Humor - Scene Annotation,Sarcasm - Scene Annotation,combination,path
0,tt2872718,0,https://www.imdb.com/videoplayer/vi4179799833,h264,854 x 480,23.976024,0,0,0,0,0000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
1,tt2872718,1,https://www.imdb.com/videoplayer/vi4179799833,h264,854 x 480,23.976024,0,0,1,0,0010,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
2,tt2788710,0,https://www.imdb.com/videoplayer/vi1114222361,h264,854 x 480,23.976024,0,0,0,0,0000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
3,tt2788710,1,https://www.imdb.com/videoplayer/vi1114222361,h264,854 x 480,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
4,tt2788710,2,https://www.imdb.com/videoplayer/vi1114222361,h264,854 x 480,23.976024,0,0,0,0,0000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...


In [5]:
# Create a dataframe which contains multiclass classification content annotations for each video scene used in the validation set.
val_df = pd.read_csv('val.csv', dtype={'combination': object}).iloc[:,1:]
val_df["path"] = val_dir + val_df["Video ID"]+ ".0" + val_df["Scene_ID"].astype(str) + ".mp4"
val_df.head()

,Video ID,Scene_ID,Video URL,Codec,Resolution,Avg Frame rate,Mature Humor - Scene Annotation,Slapstick Humor - Scene Annotation,Gory Humor - Scene Annotation,Sarcasm - Scene Annotation,combination,path
0,tt1308728,0,https://www.youtube.com/watch?v=QP9qbhTeBII,h264,640 x 360,23.975945,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
1,tt1308728,1,https://www.youtube.com/watch?v=QP9qbhTeBII,h264,640 x 360,23.975945,1,1,0,0,1100,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
2,tt1308728,2,https://www.youtube.com/watch?v=QP9qbhTeBII,h264,640 x 360,23.975945,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
3,PGuqnE35cCg,0,https://www.youtube.com/watch?v=PGuqnE35cCg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
4,PGuqnE35cCg,1,https://www.youtube.com/watch?v=PGuqnE35cCg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...


In [6]:
# Create a dataframe which contains multiclass classification content annotations for each video scene used in the test set.
test_df = pd.read_csv('test-updated.csv', dtype={'combination': object}).iloc[:,1:]
test_df["path"] = test_dir + test_df["Video ID"]+ ".0" + test_df["Scene_ID"].astype(str) + ".mp4"
test_df.head()

,Video ID,Scene_ID,Video URL,Codec,Resolution,Avg Frame rate,Mature Humor - Scene Annotation,Slapstick Humor - Scene Annotation,Gory Humor - Scene Annotation,Sarcasm - Scene Annotation,combination,path
0,tt1741243,0,https://www.imdb.com/videoplayer/vi2169153049,h264,534 x 360,29.970000,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
1,tt1741243,1,https://www.imdb.com/videoplayer/vi2169153049,h264,534 x 360,29.970000,0,1,1,0,0110,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
2,tt1723121,0,https://www.youtube.com/watch?v=O7NHfAzg7Yg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
3,tt1723121,1,https://www.youtube.com/watch?v=O7NHfAzg7Yg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...
4,tt1723121,2,https://www.youtube.com/watch?v=O7NHfAzg7Yg,h264,640 x 360,23.976024,1,0,0,0,1000,c:\Users\maitr\Desktop\COSC6373-ComputerVision...


### Data processing

How to feed videos to a neural network for training? <br>

1. Use OpenCV VideoCapture() method to read frames from videos.
2. Extract 20 frames from the videos in even sampling method.

In [7]:
# Utilities to open video files using CV2
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]

def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    count = 0
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            
            if not ret:
                break

            if count % 11 == 0:
                frame = crop_center_square(frame)
                frame = cv2.resize(frame, resize)
                frame = frame[:, :, [2, 1, 0]]
                frames.append(frame)

            count=count+1
            
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

#### Feature Extraction with pre-trained DenseNet121

In [8]:
def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

#### Label Preprocessing 

Multilabel classification



In [9]:
labels = []
for x in train_df["combination"].values:
    lst = list(map(int, x))
    arr = np.asarray(lst)
    labels.append(arr)
labels = np.reshape(labels,(len(labels),4))
print(labels.shape)

(942, 4)


In [10]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["path"].values.tolist()

    labels = []
    for x in df["combination"].values:
        lst = list(map(int, x))
        arr = np.asarray(lst)
        labels.append(arr)
    labels = np.reshape(labels,(len(labels),4))

    # `frame_features` are what we will feed to our sequence model.
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        print(path)
        # Gather all its frames and add a batch dimension.
        frames = load_video(path)

        # Pad shorter videos.
        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
            frames = np.concatenate((frames, padding))

        frames = frames[None, ...]

        # Initialize placeholder to store the features of the current video.
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )

                else:
                    temp_frame_features[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_features.squeeze()

    return frame_features, labels

In [24]:
train_data, train_labels = prepare_all_videos(train_df, "train_data")
val_data, val_labels = prepare_all_videos(val_df, "val_data")
test_data, test_labels = prepare_all_videos(test_df, "test_data")

print(f"Frame features in train set: {train_data.shape}")
print(f"Train labels: {train_labels}")

c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt2872718.00.mp4
0
11
22
33
44
55
66
77
88
99
110
121
132
143
154
165
176
187
198
209
220
231
242
253
264
275
286
297
308
319
330
341
352
363
374
385
396
407
418
429
440
451
462
473
484
495
506
517
528
539
550
561
572
583
594
605
616
627
638
649
660
671
682
693
704
715
726
737
748
759
770
781
792
803
814
825
836
847
858
869
880
891
902
913
924
935
946
957
968
979
990
1001
1012
1023
1034
1045
1056
1067
1078
1089
1100
1111
1122
1133
1144
1155
1166
1177
1188
1199
1210
1221
1232
1243
1254
1265
1276
1287
1298
1309
1320
1331
1342
1353
1364
1375
1386
1397
1408
1419
1430
1441
1452
c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/train_data/tt2872718.01.mp4
0
11
22
33
44
55
66
77
88
99
110
121
132
143
154
165
176
187
198
209
220
231
242
253
264
275
286
297
308
319
330
341
352
363
374
385
396
407
418
429
440
451
462
473
484
495
506
517
528
539
550
561
572
583
594
605


In [25]:
# Save to file
with open('train_data_3.npy', 'wb') as f:
    np.save(f, train_data)

with open('train_labels_3.npy', 'wb') as f:
    np.save(f, train_labels)

with open('val_data_3.npy', 'wb') as f:
    np.save(f, val_data)

with open('val_labels_3.npy', 'wb') as f:
    np.save(f, val_labels)

with open('test_data_3.npy', 'wb') as f:
    np.save(f, test_data)

with open('test_labels_3.npy', 'wb') as f:
    np.save(f, test_labels)

## Build the Transformer-based model (not completed) - BASE MODEL

In [11]:
# Embedding Layer
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [12]:
# Subclassed layer
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation=tf.nn.gelu), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

### Utility functions for training

In [13]:
def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 4
    num_heads = 1
    # classes = len(label_processor.get_vocabulary())
    classes = 4

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
    )
    model.summary()
    return model


def run_experiment():
    filepath = os.getcwd() + "/tmp_3/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    with tf.device('/device:CPU:0'):
        model = get_compiled_model()
        history = model.fit(
            train_data,
            train_labels,
            validation_data=(val_data,val_labels),
            epochs=EPOCHS,
            callbacks=[checkpoint],
        )

    model.load_weights(filepath)
    _, accuracy = model.evaluate(test_data, test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

## Model Training

In [14]:
train_data, train_labels = np.load("train_data_3.npy"), np.load("train_labels_3.npy")
val_data, val_labels = np.load("val_data_3.npy"), np.load("val_labels_3.npy")
test_data, test_labels = np.load("test_data_3.npy"), np.load("test_labels_3.npy")

print(f"Frame features in train set: {train_data.shape}")
print(train_labels.shape)

Frame features in train set: (942, 128, 1024)
(942, 4)


In [18]:
trained_model = run_experiment()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, None)]      0         
_________________________________________________________________
frame_position_embedding (Po (None, None, 1024)        131072    
_________________________________________________________________
transformer_layer (Transform (None, None, 1024)        4211716   
_________________________________________________________________
global_max_pooling1d (Global (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 4100      
Total params: 4,346,888
Trainable params: 4,346,888
Non-trainable params: 0
___________________________________________________

## Evaluate the base model

In [21]:
def prepare_single_video(frames):
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    # Pad shorter videos.
    if len(frames) < MAX_SEQ_LENGTH:
        diff = MAX_SEQ_LENGTH - len(frames)
        padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
        frames = np.concatenate((frames, padding))

    frames = frames[None, ...]
  
    # Extract features from the frames of the current video.
    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            if np.mean(batch[j, :]) > 0.0:
                frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
            else:
                frame_features[i, j, :] = 0.0

    return frame_features


In [34]:
predict = []
actual = []

for path in test_df['path']:
    frames = load_video(os.path.join("test", path))
    frame_features = prepare_single_video(frames)
    y_pred = trained_model.predict(frame_features)[0]
    
    # round probabilities to class labels
    y_pred = y_pred.round()
    predict.append(y_pred)

In [35]:
recall = recall_score(y_true=test_labels, y_pred=predict, average='weighted')
print("Recall", recall)
precision = precision_score(y_true=test_labels, y_pred=predict, average='weighted')
print("Precision", precision)
f1 = f1_score(y_true=test_labels, y_pred=predict, average='weighted')
print("F1 score", f1)

Recall 0.45054945054945056
Precision 0.7459182184039869
F1 score 0.48257920386517755


In [36]:
print(classification_report(test_labels, predict))

              precision    recall  f1-score   support

           0       0.44      0.62      0.52        24
           1       1.00      0.06      0.11        18
           2       1.00      0.17      0.29         6
           3       0.77      0.56      0.65        43

   micro avg       0.61      0.45      0.52        91
   macro avg       0.80      0.35      0.39        91
weighted avg       0.75      0.45      0.48        91
 samples avg       0.37      0.34      0.35        91



C:\Users\maitr\anaconda3\envs\tf.2.6\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\maitr\anaconda3\envs\tf.2.6\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [29]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def get_compiled_model_2():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 4
    num_heads = 1
    # classes = len(label_processor.get_vocabulary())
    classes = 4

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(classes, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    # compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])

    model.summary()
    return model


def run_experiment_3():
    log_dir = "logs/fit/tmp_3_3" 
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    filepath = os.getcwd() + "/tmp_3_3/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    with tf.device('/device:CPU:0'):
        model = get_compiled_model_2()
        history = model.fit(
            train_data,
            train_labels,
            validation_data=(val_data,val_labels),
            epochs=EPOCHS,
            callbacks=[checkpoint, tensorboard_callback],
        )

    model.load_weights(filepath)
    # _, accuracy = model.evaluate(test_data, test_labels)
    # evaluate the model
    loss, accuracy, f1_score, precision, recall = model.evaluate(test_data, test_labels, verbose=0)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"F1 score: {round(f1_score * 100, 2)}%")
    print(f"Precision: {round(precision * 100, 2)}%")
    print(f"Recall: {round(recall * 100, 2)}%")

    return model

In [30]:
new_model = run_experiment_3()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None, None)]      0         
_________________________________________________________________
frame_position_embedding (Po (None, None, 1024)        131072    
_________________________________________________________________
transformer_layer (Transform (None, None, 1024)        4211716   
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 4100      
Total params: 4,346,888
Trainable params: 4,346,888
Non-trainable params: 0
_________________________________________________

C:\Users\maitr\anaconda3\envs\tf.2.6\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


30/30 [==============================] - 25s 787ms/step - loss: 0.8146 - accuracy: 0.3163 - f1_m: 0.2375 - precision_m: 0.3105 - recall_m: 0.2154 - val_loss: 0.5362 - val_accuracy: 0.2389 - val_f1_m: 0.3159 - val_precision_m: 0.4890 - val_recall_m: 0.2478

Epoch 00001: val_loss improved from inf to 0.53623, saving model to c:\Users\maitr\Desktop\COSC6373-ComputerVision\Rating\RATING-Project\RATING-Project/tmp_3_3\video_classifier
Epoch 2/10
30/30 [==============================] - 15s 508ms/step - loss: 0.6359 - accuracy: 0.3142 - f1_m: 0.2414 - precision_m: 0.3268 - recall_m: 0.2023 - val_loss: 0.5407 - val_accuracy: 0.2832 - val_f1_m: 0.1269 - val_precision_m: 0.5750 - val_recall_m: 0.0759

Epoch 00002: val_loss did not improve from 0.53623
Epoch 3/10
30/30 [==============================] - 15s 492ms/step - loss: 0.5778 - accuracy: 0.3333 - f1_m: 0.2678 - precision_m: 0.3759 - recall_m: 0.2224 - val_loss: 0.5467 - val_accuracy: 0.2920 - val_f1_m: 0.3512 - val_precision_m: 0.4716 - v

## Visualization

In [ ]:
def predict_action(path):
    # class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features = prepare_single_video(frames)
    pred = new_model.predict(frame_features)
    # probabilities = trained_model.predict(frame_features)[0]

    # for i in np.argsort(probabilities)[::-1]:
    #     print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")

    # pred = np.argsort(probabilities)[::-1]
    print(pred)
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, fps=10)
    return embed.embed_file("animation.gif")

In [ ]:
test_video = np.random.choice(test_df["path"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = predict_action(test_video)
to_gif(test_frames[:25])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7327af42-8a03-4c46-b38e-e6931aa020f3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>